<a href="https://colab.research.google.com/github/git-ginwook/InsightToInterface/blob/main/explorations/smart_staidum/VIP_YoloWorld_playerDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ultalytics Documentation: https://docs.ultralytics.com/quickstart/#install-ultralytics
https://docs.ultralytics.com/models/yolo-world/#train-usage

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLOWorld

# # Load a pretrained YOLOv8s-worldv2 model
# model = YOLOWorld("/content/yolov8s-worldv2.pt")

# Load a pretrained YOLOv8x-worldv2 model
model = YOLOWorld("/yolov8x-worldv2.pt")

Detecting objects in image

In [ ]:
# Run inference with the YOLOv8n model on the 'bus.jpg' image
# img_results = model("test.jpg")
img_results = model("2017_G6017_Q1_01.jpeg")

In [ ]:
img_results[0].show()


Tracking objects on video

In [ ]:
# Track with a YOLO-World model on a video
video_results = model.track(source="005.mp4")

In [ ]:
# show zero-shot for the starting frame (0th frame)
video_results[0].show()

Single Frame
- filter for 'person' only

In [ ]:
# Extract results from the first frame
first_frame_result = video_results[0]

# Check if there are detections in the first frame
if first_frame_result is not None and first_frame_result.boxes is not None:
    # Extract bounding boxes, class IDs, and confidence scores
    boxes = first_frame_result.boxes.xyxy.tolist()
    class_ids = first_frame_result.boxes.cls.tolist()
    confidences = first_frame_result.boxes.conf.tolist()

    # Read original frame
    frame = first_frame_result.orig_img  # Original image from results

    # Iterate over detections and draw only 'person' bounding boxes
    for i, class_id in enumerate(class_ids):
        label = model.names[int(class_id)]  # Convert class ID to label
        if label == 'person':  # Filter only 'person' detections
            x1, y1, x2, y2 = map(int, boxes[i])  # Get bbox coordinates
            conf = confidences[i]  # Confidence score

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            text = f"{label} {conf:.2f}"
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Convert BGR (OpenCV format) to RGB (Matplotlib format)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Display the frame
    plt.figure(figsize=(10, 6))
    plt.imshow(frame_rgb)
    plt.axis("off")
    plt.title("First Frame - Person Detections")
    plt.show()
else:
    print("No detections found in the first frame.")


ALL frames
- filter for 'person' only

In [ ]:
# Define a function to filter detections for the 'person' class
def filter_person_detections(video_results):
    person_detections = []

    for result in video_results:  # Iterate through results (one per frame)
        if result is not None and result.boxes is not None:
            boxes = result.boxes  # Get bounding boxes
            class_ids = boxes.cls.tolist()  # Convert class tensor to list
            confidences = boxes.conf.tolist()  # Convert confidence tensor to list
            bboxes = boxes.xyxy.tolist()  # Convert bounding boxes tensor to list

            # Iterate over detected objects
            for i, class_id in enumerate(class_ids):
                label = model.names[int(class_id)]  # Convert class ID to label
                if label == 'person':  # Filter only 'person' detections
                    person_detections.append({
                        "bbox": bboxes[i],  # Bounding box coordinates (x1, y1, x2, y2)
                        "confidence": confidences[i],  # Confidence score
                        "class": label
                    })

    return person_detections

# Get filtered 'person' detections
person_detections = filter_person_detections(video_results)

# Print or save the results
for detection in person_detections:
    print(detection)